In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('dialogueText_196.csv')
df

,folder,dialogueID,date,from,to,text
0,301,1.tsv,2004-11-23T11:49:00.000Z,stuNNed,NaN,any ideas why java plugin takes so long to load?
1,301,1.tsv,2004-11-23T11:49:00.000Z,crimsun,stuNNed,java 1.4?
2,301,1.tsv,2004-11-23T11:49:00.000Z,stuNNed,crimsun,yes
3,301,1.tsv,2004-11-23T11:49:00.000Z,crimsun,stuNNed,java 1.5 loads _much_ faster
4,301,1.tsv,2004-11-23T11:50:00.000Z,stuNNed,crimsun,noneus: how can i get 1.5 is there a .deb some...
...,...,...,...,...,...,...
9212872,13,3676.tsv,2012-07-07T20:17:00.000Z,MonkeyDust,legolas,= arian
9212873,13,3676.tsv,2012-07-07T20:18:00.000Z,MonkeyDust,legolas,"observation and deduction, dear watson"
9212874,13,16586.tsv,2008-07-25T01:53:00.000Z,linuxfce,NaN,i am trying to install nvidia drivers from the...
9212875,13,16586.tsv,2008-07-25T01:53:00.000Z,linuxfce,NaN,how do i enter runlevel 3? when i try telinit ...


In [6]:
# Convert all values in the 'text' column to strings, replacing NaN with an empty string
df['text'] = df['text'].astype(str).fillna('')
df_combined = df.groupby('from')['text'].apply(' '.join).reset_index()
df_combined

,from,text
0,13WAA0N8G,- also post the url for where you posted too i...
1,13WABHP8E,hi all May I know how to install vim in arch l...
2,14WAADH16,..English please. Silicone if you mean Virutal...
3,16SAAEJK3,The colors on my display changed to their comp...
4,16SAAGH3E,no windows borders help please http://imagebin...
...,...,...
254903,}cAs{hawk16,yo ubuntu owns only good thing about thanksgiv...
254904,}dec3pti0n{,no such file there a /boot/grub/default that s...
254905,}i{,Can I ask for little help please? to like this...
254906,}matrix{,i want to change my mac.. i tried a lot of typ...


In [7]:
import re
import nltk
from nltk.corpus import stopwords

#Create a function that will remove HTML tags, non-alphanumeric characters, and symbols using regex, and remove stopwords with nltk
def clean_text(text):
    #Remove HTML tags
    ct = re.sub('<.*?>', '', text)
    
    #Remove non-alphanumeric characters and symbols
    ct = re.sub(r'[^a-zA-Z\s]', '', ct)
    
    #Convert to lowercase
    #ct = ct.lower()
    
    #Import stop words from the English language
    stop_words = set(stopwords.words('english'))
    #Tokenize the current review text
    words = nltk.word_tokenize(ct)
    #Keep word if it's not in stop words that were imported
    filtered_words = [word for word in words if word.lower() not in stop_words]
    #Return cleaned string
    return ' '.join(filtered_words)

#Apply the function to each row of the DataFrame to clean the data
df_combined['text'] = df_combined['text'].apply(clean_text)
df_combined

In [9]:
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
import sklearn.metrics as metrics
import pickle

# Load the model from the file
with open('xgboost_model.pkl', 'rb') as model_file:
    loaded_model = pickle.load(model_file)
    
with open('tfidf_vectorizer.pkl', 'rb') as vectorizer_file:
    loaded_vectorizer = pickle.load(vectorizer_file)

#Prepare testing data
X_test = df_combined['text']

# Feature extraction using TF-IDF (using the same fitted vectorizer)
X_test_tfidf = loaded_vectorizer.transform(X_test)

# Use the loaded model to make predictions
y_pred = loaded_model.predict(X_test_tfidf)

y_pred = y_pred.astype(int)

In [10]:
df_combined['MBTI Type'] = y_pred
df_combined

,from,text,MBTI Type
0,13WAA0N8G,also post url posted helps,4
1,13WABHP8E,hi May know install vim arch linux hi May know...,4
2,14WAADH16,English please Silicone mean Virutal Machine u...,4
3,16SAAEJK3,colors display changed complement eg whiteblac...,4
4,16SAAGH3E,windows borders help please httpimagebinorg pl...,4
...,...,...,...
254903,}cAs{hawk16,yo ubuntu owns good thing thanksgiving food su...,4
254904,}dec3pti0n{,file bootgrubdefault says use grubsetdefault see,4
254905,}i{,ask little help please like page sell hamsters...,12
254906,}matrix{,want change mac tried lot typs dont works mac ...,4


In [69]:
import pandas as pd

# Sample 10 rows with MBTI type 4
sampled_type_4 = df_combined[df_combined['MBTI Type'] == 4].sample(n=10, random_state=42)

# Sample 5 rows from the remaining MBTI types
sampled_other_types = df_combined[df_combined['MBTI Type'] != 4].sample(n=5, random_state=42)

# Combine the samples
sampled_df = pd.concat([sampled_type_4, sampled_other_types])

# Shuffle the resulting DataFrame (optional)
sampled_df = sampled_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Display the sampled DataFrame
sampled_df


,from,text,MBTI Type
0,blinkija,talking talking ubuntu,4
1,Lad,check kernal im running upgrade atheros ar chi...,8
2,[set],ncmpc mpd ncmpc,4
3,ricksta,would recommend ubuntu xubuntu computer like m...,0
4,Un-Dead,anyone helps dont belive hardware issue window...,4
5,[StingeR],alraune Russish alraune Russish,4
6,Samuusi,ask change language firefox ty errr zvacet wha...,4
7,BoKakSi,actualy key sequence try ctrls console stop ca...,4
8,s0me_schmuck,Spliff loved able make gentoo linux stage usin...,0
9,CeruM,hi anyone help install dont resize partition o...,4


In [71]:
sampled_df['text'].to_list()

['talking talking ubuntu',
 'check kernal im running upgrade atheros ar chipset works box upgrade kernel karma alpha install athos wifi driver thats usb recgnized goto website ctmjr nvidia card installed Ubuntu desktop second harddrive im going dropping vista going ubuntu curious boot install get wired internet windows works Im sure begin help sorry cant get wired internet correct nVidia LT SLI q gb ram gb sata gt however lose connection wait minutes return output steps take time someone help deleting panels anyone able teach delete panels lol dosenthae option remoe cusomize add items quit restart anyone able helpme delete panel right click remove opton options arecustomize add item quit restart helpmelol hi someone help delete panel actually dont gnome panels open floating around screen think used panel manger add clicked plus sign dont see gnome settings anywhere dont think xubuntu fresh still im nt sure ubuntu better xubuntu panels delete options anywhere good machiene',
 'ncmpc mpd

In [73]:
chat_labels = [5, 0, 5, 5, 0, 4, 5, 1, 0, 9, 0, 0, 9, 6, 0]

In [74]:
sampled_df['Chat-GPT Labeling'] = chat_labels
sampled_df

,from,text,MBTI Type,Chat-GPT Labeling
0,blinkija,talking talking ubuntu,4,5
1,Lad,check kernal im running upgrade atheros ar chi...,8,0
2,[set],ncmpc mpd ncmpc,4,5
3,ricksta,would recommend ubuntu xubuntu computer like m...,0,5
4,Un-Dead,anyone helps dont belive hardware issue window...,4,0
5,[StingeR],alraune Russish alraune Russish,4,4
6,Samuusi,ask change language firefox ty errr zvacet wha...,4,5
7,BoKakSi,actualy key sequence try ctrls console stop ca...,4,1
8,s0me_schmuck,Spliff loved able make gentoo linux stage usin...,0,0
9,CeruM,hi anyone help install dont resize partition o...,4,9


In [77]:
sampled_df['MBTI Text XGB'] = sampled_df['MBTI Type'].replace(
    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    ['INTP', 'ISTP', 'ENTP', 'ESTP', 'INFP', 'ISFP', 'ENFP', 'ESFP', 
     'INTJ', 'ISTJ', 'ENTJ', 'ESTJ', 'INFJ', 'ISFJ', 'ENFJ', 'ESFJ']
)

sampled_df['MBTI Text Chat-GPT'] = sampled_df['Chat-GPT Labeling'].replace(
    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    ['INTP', 'ISTP', 'ENTP', 'ESTP', 'INFP', 'ISFP', 'ENFP', 'ESFP', 
     'INTJ', 'ISTJ', 'ENTJ', 'ESTJ', 'INFJ', 'ISFJ', 'ENFJ', 'ESFJ']
)

In [83]:
#Function to check for 'I' or 'E'
def check_IE(text):
    if 'I' in text:
        return 0
    elif 'E' in text:
        return 1

#Function to check for 'N' or 'S'
def check_NS(text):
    if 'N' in text:
        return 0
    elif 'S' in text:
        return 1

#Function to check for 'F' or 'T'  
def check_FT(text):
    if 'F' in text:
        return 0
    elif 'T' in text:
        return 1
    
#Function to check for 'J' or 'P'
def check_JP(text):
    if 'J' in text:
        return 0
    elif 'P' in text:
        return 1

#Apply the function to the 'type' column
sampled_df['XGB IE'] = sampled_df['MBTI Text XGB'].apply(check_IE)
sampled_df['XGB NS'] = sampled_df['MBTI Text XGB'].apply(check_NS)
sampled_df['XGB FT'] = sampled_df['MBTI Text XGB'].apply(check_FT)
sampled_df['XGB JP'] = sampled_df['MBTI Text XGB'].apply(check_JP)

#Apply the function to the 'type' column
sampled_df['GPT IE'] = sampled_df['MBTI Text Chat-GPT'].apply(check_IE)
sampled_df['GPT NS'] = sampled_df['MBTI Text Chat-GPT'].apply(check_NS)
sampled_df['GPT FT'] = sampled_df['MBTI Text Chat-GPT'].apply(check_FT)
sampled_df['GPT JP'] = sampled_df['MBTI Text Chat-GPT'].apply(check_JP)

sampled_df.head()

,from,text,MBTI Type,Chat-GPT Labeling,MBTI Text XGB,MBTI Text Chat-GPT,XGB IE,XGB NS,XGB FT,XGB JP,GPT IE,GPT NS,GPT FT,GPT JP
0,blinkija,talking talking ubuntu,4,5,INFP,ISFP,0,0,0,1,0,1,0,1
1,Lad,check kernal im running upgrade atheros ar chi...,8,0,INTJ,INTP,0,0,1,0,0,0,1,1
2,[set],ncmpc mpd ncmpc,4,5,INFP,ISFP,0,0,0,1,0,1,0,1
3,ricksta,would recommend ubuntu xubuntu computer like m...,0,5,INTP,ISFP,0,0,1,1,0,1,0,1
4,Un-Dead,anyone helps dont belive hardware issue window...,4,0,INFP,INTP,0,0,0,1,0,0,1,1


In [84]:
import pandas as pd

def calculate_dichotomy_similarity(row):
    """Calculate the similarity percentage between XGB and GPT dichotomies."""
    xgb_dichotomies = row[['XGB IE', 'XGB NS', 'XGB FT', 'XGB JP']].values
    gpt_dichotomies = row[['GPT IE', 'GPT NS', 'GPT FT', 'GPT JP']].values
    
    # Calculate the number of matching dichotomies
    matches = (xgb_dichotomies == gpt_dichotomies).sum()
    total_dichotomies = len(xgb_dichotomies)
    
    # Calculate the similarity percentage
    similarity_percentage = (matches / total_dichotomies) * 100
    return similarity_percentage

# Apply the similarity function
sampled_df['dichotomy_similarity'] = sampled_df.apply(calculate_dichotomy_similarity, axis=1)

In [85]:
sampled_df

,from,text,MBTI Type,Chat-GPT Labeling,MBTI Text XGB,MBTI Text Chat-GPT,XGB IE,XGB NS,XGB FT,XGB JP,GPT IE,GPT NS,GPT FT,GPT JP,dichotomy_similarity
0,blinkija,talking talking ubuntu,4,5,INFP,ISFP,0,0,0,1,0,1,0,1,75.0
1,Lad,check kernal im running upgrade atheros ar chi...,8,0,INTJ,INTP,0,0,1,0,0,0,1,1,75.0
2,[set],ncmpc mpd ncmpc,4,5,INFP,ISFP,0,0,0,1,0,1,0,1,75.0
3,ricksta,would recommend ubuntu xubuntu computer like m...,0,5,INTP,ISFP,0,0,1,1,0,1,0,1,50.0
4,Un-Dead,anyone helps dont belive hardware issue window...,4,0,INFP,INTP,0,0,0,1,0,0,1,1,75.0
5,[StingeR],alraune Russish alraune Russish,4,4,INFP,INFP,0,0,0,1,0,0,0,1,100.0
6,Samuusi,ask change language firefox ty errr zvacet wha...,4,5,INFP,ISFP,0,0,0,1,0,1,0,1,75.0
7,BoKakSi,actualy key sequence try ctrls console stop ca...,4,1,INFP,ISTP,0,0,0,1,0,1,1,1,50.0
8,s0me_schmuck,Spliff loved able make gentoo linux stage usin...,0,0,INTP,INTP,0,0,1,1,0,0,1,1,100.0
9,CeruM,hi anyone help install dont resize partition o...,4,9,INFP,ISTJ,0,0,0,1,0,1,1,0,25.0


In [88]:
# Evaluate model performance on validation data
print("Validation Classification Report:")
print(classification_report(sampled_df['MBTI Type'], sampled_df['Chat-GPT Labeling']))

Validation Classification Report:
              precision    recall  f1-score   support

           0       0.17      0.50      0.25         2
           1       0.00      0.00      0.00         0
           4       1.00      0.10      0.18        10
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         2
           9       0.00      0.00      0.00         0
          12       0.00      0.00      0.00         1

    accuracy                           0.13        15
   macro avg       0.15      0.07      0.05        15
weighted avg       0.69      0.13      0.15        15



c:\Users\patog\miniconda3\envs\torchy\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\patog\miniconda3\envs\torchy\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\patog\miniconda3\envs\torchy\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\